In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
!pip install -qq colorama
!pip install -qq timm
import os, shutil

#必要なライブラリのインポート
import re, gc, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

import warnings, random
import cv2

warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler

import torchvision
from torchvision import transforms
import torchvision.models as models
from torch.cuda.amp import GradScaler

import timm
import yaml
from tqdm import tqdm
import time
import copy
from collections import defaultdict

from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

In [ ]:
!nvidia-smi

Fri Nov  1 11:41:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              47W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

画像データの読み込み

In [ ]:
DATA_ROOT = Path("/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou")
INPUT = DATA_ROOT/"input"
OUTPUT = DATA_ROOT/"output"

In [ ]:
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

fix_all_seeds(2021)

In [ ]:
!cp -r "/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/input/train.zip" .
!cp -r "/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/input/test.zip" .

In [ ]:
# ファイルの解凍
!unzip "/content/train.zip"
!unzip "/content/test.zip"

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: test/TF67rDzGkjGxUMR5WKkaS8.jpg  
  inflating: test/Tf8ShD3qUCgADTRTu5ronj.jpg  
  inflating: test/TF8zj9FM4dGV4hJ68PfVbm.jpg  
  inflating: test/Tf9Vp3RhQB8gTKgLR9BBC8.jpg  
  inflating: test/TFbESAmi5KQ2bCaYsRvQxv.jpg  
  inflating: test/TfcRNTXVNQHG3ymcodvr89.jpg  
  inflating: test/TfeZsarrD7L7nQKd6teNFC.jpg  
  inflating: test/TFgaFdgqNu55PBNTLYfygX.jpg  
  inflating: test/TfiW2x5m3Y65t5vf6kkiwP.jpg  
  inflating: test/TFj62G9peUkRFc7CNnSAKW.jpg  
  inflating: test/TFKBKekvDXWvMuRjTv2jw7.jpg  
  inflating: test/TfKPdqXZkATQ8njVkwiCvi.jpg  
  inflating: test/TfNcFZaz7Cut2AdJTGxTKh.jpg  
  inflating: test/TFNF6LaDBhFTfBfwE44hgS.jpg  
  inflating: test/TFoDSSgXjaSc4dnaGGhSLw.jpg  
  inflating: test/TfoffMJicMqZeFFPX53sFH.jpg  
  inflating: test/TFou8VboCCP5QNDw5NLXnA.jpg  
  inflating: test/TFqGk6GaQ5NnAD3bRWS5nW.jpg  
  inflating: test/TFsTJLAN9zmLRrVgbdxCz4.jpg  
  inflating: test/TfU78JPgJcNzg355BNx9is.jpg  
  inflating: test/TfuaVNoZSA

In [ ]:
TRAIN_HOLD_PATH = Path("/content/train/hold/")
TRAIN_NOTHOLD_PATH = Path("/content/train/not-hold/")

TEST_PATH = Path("/content/test")

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
classifications = ["not-hold a folding fan", "hold a folding fan"]

def search_images(directory):
    # 対応する画像の拡張子を定義
    image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')

    # 画像ファイルのパスとファイル名をリストに格納
    image_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(image_extensions):
                file_path = os.path.join(root, file)
                image_files.append({
                    'file_name': file,
                    'file_path': file_path
                })

    # pandas DataFrame に変換
    df = pd.DataFrame(image_files)

    return df

#HOME = Path("/content")
#INPUTS = HOME / "dataset"  # input data
INPUTS = Path("/content")
TRAIN_IMAGEDIR0 = INPUTS / "train" / "not-hold"
TRAIN_IMAGEDIR1 = INPUTS / "train" / "hold"

train_df = pd.DataFrame()
train_df0 = search_images(TRAIN_IMAGEDIR0)
train_df1 = search_images(TRAIN_IMAGEDIR1)
train_df0['caption'] = classifications[0]
train_df1['caption'] = classifications[1]
train_df0['label'] = 0
train_df1['label'] = 1
train_df = pd.concat([train_df0, train_df1], axis=0)
print(train_df)
train_df.to_csv('/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/input/train.csv', index=None)

                        file_name  \
0      PrdshfoK8KmTqok2ff5zZ9.jpg   
1      EeksEjXNjP5TnFnxBKgTZp.jpg   
2      TMsUDVu6UxNhsEQS4N6YeX.jpg   
3      iYsm6vbCKtwaR72CVfGnZs.jpg   
4      jebvuow5WgY7VZKMFzDGxB.jpg   
...                           ...   
34467  oVcFXBKAqpQPgcNSUqSrY7.jpg   
34468  deFYh8WHUWZY9v46LmW4jd.jpg   
34469  NgPHQyPN5K9ghXZkhLGFGF.jpg   
34470  hN8kgEKDwA9XLTZXLZeYGz.jpg   
34471  4YeJVfAxdN7jLLfnT9G9M6.jpg   

                                               file_path  \
0      /content/train/not-hold/PrdshfoK8KmTqok2ff5zZ9...   
1      /content/train/not-hold/EeksEjXNjP5TnFnxBKgTZp...   
2      /content/train/not-hold/TMsUDVu6UxNhsEQS4N6YeX...   
3      /content/train/not-hold/iYsm6vbCKtwaR72CVfGnZs...   
4      /content/train/not-hold/jebvuow5WgY7VZKMFzDGxB...   
...                                                  ...   
34467     /content/train/hold/oVcFXBKAqpQPgcNSUqSrY7.jpg   
34468     /content/train/hold/deFYh8WHUWZY9v46LmW4jd.jpg   
34469     /cont

In [ ]:
avail_pretrained_models = timm.list_models(pretrained=True)
avail_pretrained_models

['bat_resnext26ts.ch_in1k',
 'beit_base_patch16_224.in22k_ft_in22k',
 'beit_base_patch16_224.in22k_ft_in22k_in1k',
 'beit_base_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_224.in22k_ft_in22k',
 'beit_large_patch16_224.in22k_ft_in22k_in1k',
 'beit_large_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_512.in22k_ft_in22k_in1k',
 'beitv2_base_patch16_224.in1k_ft_in1k',
 'beitv2_base_patch16_224.in1k_ft_in22k',
 'beitv2_base_patch16_224.in1k_ft_in22k_in1k',
 'beitv2_large_patch16_224.in1k_ft_in1k',
 'beitv2_large_patch16_224.in1k_ft_in22k',
 'beitv2_large_patch16_224.in1k_ft_in22k_in1k',
 'botnet26t_256.c1_in1k',
 'caformer_b36.sail_in1k',
 'caformer_b36.sail_in1k_384',
 'caformer_b36.sail_in22k',
 'caformer_b36.sail_in22k_ft_in1k',
 'caformer_b36.sail_in22k_ft_in1k_384',
 'caformer_m36.sail_in1k',
 'caformer_m36.sail_in1k_384',
 'caformer_m36.sail_in22k',
 'caformer_m36.sail_in22k_ft_in1k',
 'caformer_m36.sail_in22k_ft_in1k_384',
 'caformer_s18.sail_in1k',
 'caformer_s18.s

分析の設定

In [ ]:
ARGS = {
  'DATA_DIR': '/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/input',
  'OUT_DIR': '/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/output',
  'model_name': 'vit_l_16',
  # vit_l16
  #cpu(slow)
  #'train_batch_size': 4,
  #'test_batch_size': 8,
  #gpu
  #'train_batch_size': 28, # 32(x)
  #'test_batch_size': 56,
  #'n_fold': 2,
  #'epochs': 3,
  #'timm_model_name': 'resnet50',
  #'timm_model_name': 'vit_base_patch16_224',
  #lb:0.8545424 ? :
  #'timm_model_name': 'vit_large_patch32_224_in21k',
  #down?
  #'timm_model_name': 'vit_huge_patch14_224_in21k',
  #colab free crash memory?.(batch=4)
  #'timm_model_name': 'vit_giant_patch14_224_clip_laion2b',
  #Only one class present in y_true. ROC AUC score is not defined in that case.
  #lb:0.8944379  : top, batchsize : 4, 'image_size': (448, 448),
  'timm_model_name': 'eva02_large_patch14_448.mim_m38m_ft_in22k_in1k',
  'pretrained': True,
  'n_fold': 5, # 5
  'epochs': 4, # 8
  'image_size': (448, 448), # eva02_large_patch14_448
  'criterion': 'CrossEntropy',
  #'is_blurry': True,
  'is_blurry': False,
  #'image_size': (336, 336),
  #GPU: 16GB
  'train_batch_size': 8, # 4, #1(ng?)
  'test_batch_size': 16, #x3?
  #GPU: 80GB?
  #'train_batch_size': 32, # 4, #1(ng?)
  #'test_batch_size': 96,
  #batchsize row is roc_auc calc ng,(batchsizeが小さいと、class data が片方のみ存在している状態になり roc_auc の計算ができない)
  'seed': 2023,
  'optimizer': 'AdamW',
  'learning_rate': 1e-05,
  'scheduler': 'CosineAnnealingLR', # CosineAnnealingWarmRestarts
  'min_lr': 1e-06,
  'T_max': 500,
  'n_accumulate': 1,
  'clip_grad_norm': 'None',
  'apex': True,
  'num_classes': 2,
  'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  }


画像名、パス、ラベル、kfold数に対応する表の作成

In [ ]:
def create_folds(data, num_splits, seed):
    data["kfold"] = -1

    mskf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=seed)
    labels = ["label"]
    data_labels = data[labels].values

    for f, (t_, v_) in enumerate(mskf.split(data, data_labels)):
        data.loc[v_, "kfold"] = f

    return data

train = pd.read_csv(f"{ARGS['DATA_DIR']}/train.csv")
train = create_folds(train, num_splits=ARGS["n_fold"], seed=ARGS["seed"])
print("Folds created successfully")

train.head()

Folds created successfully


,file_name,file_path,caption,label,kfold
0,PrdshfoK8KmTqok2ff5zZ9.jpg,/content/train/not-hold/PrdshfoK8KmTqok2ff5zZ9...,not-hold a folding fan,0,1
1,EeksEjXNjP5TnFnxBKgTZp.jpg,/content/train/not-hold/EeksEjXNjP5TnFnxBKgTZp...,not-hold a folding fan,0,4
2,TMsUDVu6UxNhsEQS4N6YeX.jpg,/content/train/not-hold/TMsUDVu6UxNhsEQS4N6YeX...,not-hold a folding fan,0,2
3,iYsm6vbCKtwaR72CVfGnZs.jpg,/content/train/not-hold/iYsm6vbCKtwaR72CVfGnZs...,not-hold a folding fan,0,0
4,jebvuow5WgY7VZKMFzDGxB.jpg,/content/train/not-hold/jebvuow5WgY7VZKMFzDGxB...,not-hold a folding fan,0,1


画像の前処理、データ拡張

In [ ]:
def get_logger(filename):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler2)
    return logger

#再現性を出すために必要な関数となります
def worker_init_fn(worker_id):
    torch.manual_seed(worker_id)
    random.seed(worker_id)
    np.random.seed(worker_id)
    torch.cuda.manual_seed(worker_id)
    os.environ['PYTHONHASHSEED'] = str(worker_id)

def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)


LOGGER = get_logger(ARGS['OUT_DIR']+'/train')
set_seed(ARGS["seed"])

学習モデルの作成

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, transform, data_type):
        self.df = df
        self.data_type = data_type

        if self.data_type == "train":
            self.image_paths = df['file_path']
            self.labels = df['label']
        if self.data_type == "test":
            #self.image_paths = df[0]
            self.image_paths = df['image_path']

        self.transform= transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index: int):
        image_path = self.image_paths[index]

        if self.data_type == "train":
            #image = cv2.imread(f"/content/train/{file_name}")
            image = cv2.imread(f"{image_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            label = self.labels[index]
            label = torch.tensor(label, dtype=torch.long)

            image = self.transform(image=image)["image"]
            return image, label

        if self.data_type == "test":
            if os.path.exists(f"/content/test/{image_path}"):
              image = cv2.imread(f"/content/test/{image_path}")
              image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
              image = self.transform(image=image)["image"]
            else:
              print("not found " + image_path)
              image = None

            return image

In [ ]:
class CustomDataset2(Dataset):
    def __init__(self, df, transform, data_type, is_blurry):
        self.df = df
        self.data_type = data_type

        if self.data_type == "train":
            if is_blurry:
                self.image_paths = df['file_path']
            else:
                self.image_paths = df['file_path']
            self.labels = df['label']
        if self.data_type == "test":
            #self.image_paths = df[0]
            self.image_paths = df['image_path']

        self.transform= transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index: int):
        image_path = self.image_paths[index]

        if self.data_type == "train":
            image = cv2.imread(f"/content/train/{image_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            label = self.labels[index]
            label = torch.tensor(label, dtype=torch.long)

            image = self.transform(image=image)["image"]
            return image, label

        if self.data_type == "test":
            image = cv2.imread(f"/content/test/{image_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            image = self.transform(image=image)["image"]

            return image

In [ ]:
import albumentations as albu
from albumentations.pytorch import transforms as AT

# Augumentation用
#CV : up, LB : down
image_transform_train = albu.Compose([
    albu.Resize(ARGS["image_size"][0], ARGS["image_size"][1]),
    albu.HorizontalFlip(p=0.5),
    albu.VerticalFlip(p=0.5),
    albu.RandomBrightnessContrast(p=0.3),
    albu.RandomGamma(gamma_limit=(85, 115), p=0.3),
    albu.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.10, rotate_limit=45, p=0.5),
    albu.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    AT.ToTensorV2()
])

image_transform = albu.Compose([
    albu.Resize(ARGS["image_size"][0], ARGS["image_size"][1]),
    # albu.HorizontalFlip(p=0.5),
    # albu.VerticalFlip(p=0.5),
    # albu.RandomBrightnessContrast(p=0.3),
    # albu.RandomGamma(gamma_limit=(85, 115), p=0.3),
    # albu.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.10, rotate_limit=45, p=0.5),
    albu.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    AT.ToTensorV2()
])

In [ ]:
def train_one_epoch(model, optimizer, train_loader, device, epoch):
    model.train()
    dataset_size = 0
    running_loss = 0.0
    running_score = []
    running_score_y = []
    scaler = GradScaler(enabled=ARGS["apex"])

    train_loss = []
    bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, (images, targets) in bar:
      images = images.to(device)
      targets = targets.to(device)

      batch_size = targets.size(0)
      with torch.cuda.amp.autocast(enabled=ARGS["apex"]):
          outputs = model(images)
          loss = criterion(ARGS, outputs, targets)

      scaler.scale(loss).backward()

      if ARGS["clip_grad_norm"] != "None":
          grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), ARGS["clip_grad_norm"])
      else:
          grad_norm = None

      scaler.step(optimizer)
      scaler.update()

      optimizer.zero_grad()

      if scheduler is not None:
          scheduler.step()

      train_loss.append(loss.item())

      running_loss += (loss.item() * batch_size)
      dataset_size += batch_size

      epoch_loss = running_loss / dataset_size

      running_score.append(outputs.detach().cpu().numpy())
      running_score_y.append(targets.detach().cpu().numpy())

      score = get_score(running_score_y, running_score)

      bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                      Train_Acc=score[0],
                      Train_Auc=score[1],
                      LR=optimizer.param_groups[0]['lr']
                      )
    gc.collect()
    return epoch_loss, score


In [ ]:
@torch.no_grad()
def valid_one_epoch(args, model, optimizer, valid_loader, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0.0
    preds = []
    valid_targets = []
    softmax = nn.Softmax()

    bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
    for step, (images, targets) in enumerate(valid_loader):
      images = images.to(args["device"])
      targets = targets.to(args["device"])
      batch_size = targets.size(0)
      with torch.no_grad():
        outputs = model(images)
        predict = outputs.softmax(dim=1)
        loss = criterion(args, outputs, targets)

      running_loss += (loss.item() * batch_size)
      dataset_size += batch_size

      epoch_loss = running_loss / dataset_size

      preds.append(predict.detach().cpu().numpy())
      valid_targets.append(targets.detach().cpu().numpy())

      if len(set(np.concatenate(valid_targets))) == 1:
          continue
      score = get_score(valid_targets, preds)

      bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                      Valid_Acc=score[0],
                      Valid_Auc=score[1],
                      LR=optimizer.param_groups[0]['lr'])

    return epoch_loss, preds, valid_targets, score

In [ ]:
def one_fold(model, optimizer, schedulerr, device, num_epochs, fold):

    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_score = np.inf
    best_prediction = None

    best_score = -np.inf
    for epoch in range(1, 1+num_epochs):
      train_epoch_loss, train_score = train_one_epoch(model, optimizer,
                                              train_loader=train_loader,
                                              device=device, epoch=epoch)

      train_acc, train_auc = train_score

      val_epoch_loss, predictions, valid_targets, valid_score = valid_one_epoch(ARGS,
                                                                                model,
                                                                                optimizer,
                                                                                valid_loader,
                                                                                epoch=epoch)
      valid_acc, valid_auc = valid_score

      LOGGER.info(f'Epoch {epoch} - avg_train_loss: {train_epoch_loss:.4f}  avg_val_loss: {val_epoch_loss:.4f}')
      LOGGER.info(f'Epoch {epoch} - Train Acc: {train_acc:.4f}  Train Auc: {train_auc:.4f}  Valid Acc: {valid_acc:.4f}  Valid Auc: {valid_auc:.4f}')

      if valid_auc >= best_score:
        best_score = valid_auc

        print(f"{b_}Validation Score Improved ({best_epoch_score} ---> {valid_auc})")
        best_epoch_score = valid_auc
        best_model_wts = copy.deepcopy(model.state_dict())
        # PATH = f"Score-Fold-{fold}.bin"
        PATH = ARGS["OUT_DIR"] + f"/Score-Fold-{fold}.bin"
        torch.save(model.state_dict(), PATH)
        # Save a model file from the current directory
        print(f"Model Saved{sr_}")

        best_prediction = np.concatenate(predictions, axis=0)[:,1]

    end = time.time()
    time_elapsed = end - start

    LOGGER.info('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    LOGGER.info("Best Score: {:.4f}".format(best_epoch_score))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_prediction, valid_targets

In [ ]:
def create_model(args):
    model = models.vit_l_16(pretrained=True)
    model.heads[0] = torch.nn.Linear(in_features=model.heads[0].in_features, out_features=args["num_classes"], bias=True)
    return model

def create_model_timm(args):
    #model = timm.create_model(args["timm_model_name"], pretrained=True, num_classes=args["num_classes"])
    model = timm.create_model(args["timm_model_name"], args["pretrained"], num_classes=args["num_classes"])
    # 重みを更新するパラメータを選択する
    # 最終層だけでOK
    """
    params_to_update = []
    update_param_names = ['head.weight', 'head.bias']

    for name, param in model.named_parameters():
        if name in update_param_names:
            param.requires_grad = True
            params_to_update.append(param)
        else:
            param.requires_grad = False
    """

    return model

def criterion(args, outputs, labels, class_weights=None):
    if args['criterion'] == 'CrossEntropy':
      return nn.CrossEntropyLoss(weight=class_weights).to(args["device"])(outputs, labels)
    elif args['criterion'] == "None":
        return None

def fetch_optimizer(optimizer_parameters, lr, betas, optimizer_name="Adam"):
    if optimizer_name == "Adam":
        optimizer = optim.Adam(optimizer_parameters, lr=lr)
    elif optimizer_name == "AdamW":
        optimizer = optim.AdamW(optimizer_parameters, lr=lr, betas=betas)
    return optimizer

def fetch_scheduler(args, train_size, optimizer):

    if args['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=args['T_max'],
                                                   eta_min=args['min_lr'])
    elif args['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=args['T_0'],
                                                             eta_min=args['min_lr'])
    elif args['scheduler'] == "None":
        scheduler = None

    return scheduler

def get_score(y_trues, y_preds):
    predict_list, targets_list = np.concatenate(y_preds, axis=0), np.concatenate(y_trues)
    predict_list_proba = predict_list.copy()[:, 1]
    predict_list = predict_list.argmax(axis=1)

    accuracy = accuracy_score(predict_list, targets_list)
    try:
        auc_score = roc_auc_score(targets_list, predict_list_proba)
    except:
        auc_score = 0.0

    return (accuracy, auc_score)

def prepare_loaders(args, train, image_transform, fold):
    df_train = train[train.kfold != fold].reset_index(drop=True)
    df_valid = train[train.kfold == fold].reset_index(drop=True)

    train_dataset = CustomDataset(df_train, image_transform, data_type="train")
    valid_dataset = CustomDataset(df_valid, image_transform, data_type="train")
    #train_dataset = CustomDataset2(df_train, image_transform, data_type="train", is_blurry=ARGS['is_blurry'])
    #valid_dataset = CustomDataset2(df_valid, image_transform, data_type="train", is_blurry=ARGS['is_blurry'])

    train_loader = DataLoader(train_dataset, batch_size=args['train_batch_size'],
                              worker_init_fn=worker_init_fn(args["seed"]),
                              num_workers=4,
                              shuffle=True, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=args['test_batch_size'],
                              num_workers=4,
                              shuffle=False, pin_memory=True)

    return train_loader, valid_loader

学習の実行をするセル

In [ ]:
train_copy = train.copy()
LOGGER.info(ARGS)
for fold in range(0, ARGS['n_fold']):
    print(f"{y_}====== Fold: {fold} ======{sr_}")
    LOGGER.info(f"========== fold: {fold} training ==========")

    # Create Dataloaders
    train_loader, valid_loader = prepare_loaders(args=ARGS, train=train, image_transform=image_transform, fold=fold)
    #train_loader, valid_loader = prepare_loaders(args=ARGS, train=train, image_transform=image_transform_train, fold=fold)

    #vit_l16
    #model = create_model(ARGS)
    #heavy
    model = create_model_timm(ARGS)
    model = model.to(ARGS["device"])

    #損失関数・最適化関数の定義
    optimizer = fetch_optimizer(model.parameters(), optimizer_name=ARGS["optimizer"], lr=ARGS["learning_rate"], betas=(0.9, 0.999))

    scheduler = fetch_scheduler(args=ARGS, train_size=len(train_loader), optimizer=optimizer)

    model, predictions, targets = one_fold(model, optimizer, scheduler, device=ARGS["device"], num_epochs=ARGS["epochs"], fold=fold)

    print(predictions)
    train_copy.loc[train_copy[train_copy.kfold == fold].index, "oof"] = predictions
    #train_copy.loc[train_copy[train_copy.kfold == fold].index, "pred_0"] = predictions[:,0]
    #train_copy.loc[train_copy[train_copy.kfold == fold].index, "pred_1"] = predictions[:,1]

    del model, train_loader, valid_loader
    _ = gc.collect()
    torch.cuda.empty_cache()
    print()

scores = roc_auc_score(train_copy["label"].values, train_copy["oof"].values)
LOGGER.info(f"========== CV ==========")
LOGGER.info(f"CV: {scores:.4f}")

INFO:__main__:{'DATA_DIR': '/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/input', 'OUT_DIR': '/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/output', 'model_name': 'vit_l_16', 'image_size': (448, 448), 'timm_model_name': 'eva02_large_patch14_448.mim_m38m_ft_in22k_in1k', 'pretrained': True, 'n_fold': 5, 'epochs': 4, 'criterion': 'CrossEntropy', 'is_blurry': False, 'train_batch_size': 8, 'test_batch_size': 16, 'seed': 2023, 'optimizer': 'AdamW', 'learning_rate': 1e-05, 'scheduler': 'CosineAnnealingLR', 'min_lr': 1e-06, 'T_max': 500, 'n_accumulate': 1, 'clip_grad_norm': 'None', 'apex': True, 'num_classes': 2, 'device': device(type='cuda', index=0)}
INFO:__main__:========== fold: 0 training ==========


====== Fold: 0 ======


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

[INFO] Using GPU: NVIDIA A100-SXM4-40GB



  0%|          | 0/844 [10:21<?, ?it/s, Epoch=1, LR=5.33e-6, Valid_Acc=0.997, Valid_Auc=1, Valid_Loss=0.00832]
INFO:__main__:Epoch 1 - avg_train_loss: 0.0409  avg_val_loss: 0.0083
INFO:__main__:Epoch 1 - Train Acc: 0.9875  Train Auc: 0.9988  Valid Acc: 0.9973  Valid Auc: 0.9998


...

  0%|          | 0/844 [10:20<?, ?it/s, Epoch=2, LR=1.01e-6, Valid_Acc=0.998, Valid_Auc=1, Valid_Loss=0.00747]
INFO:__main__:Epoch 2 - avg_train_loss: 0.0133  avg_val_loss: 0.0075
INFO:__main__:Epoch 2 - Train Acc: 0.9957  Train Auc: 0.9998  Valid Acc: 0.9978  Valid Auc: 0.9999


Validation Score Improved (0.9998365248650465 ---> 0.9999478067278514)
Model Saved


  0%|          | 0/844 [10:20<?, ?it/s, Epoch=3, LR=6.01e-6, Valid_Acc=0.997, Valid_Auc=1, Valid_Loss=0.0099]
INFO:__main__:Epoch 3 - avg_train_loss: 0.0080  avg_val_loss: 0.0099
INFO:__main__:Epoch 3 - Train Acc: 0.9973  Train Auc: 0.9999  Valid Acc: 0.9966  Valid Auc: 0.9999
  0%|          | 0/844 [10:20<?, ?it/s, Epoch=4, LR=9.95e-6, Valid_Acc=0.998, Valid_Auc=1, Valid_Loss=0.00893]
INFO:__main__:Epoch 4 - avg_train_loss: 0.0059  avg_val_loss: 0.0089
INFO:__main__:Epoch 4 - Train Acc: 0.9983  Train Auc: 0.9999  Valid Acc: 0.9984  Valid Auc: 0.9999
INFO:__main__:Training complete in 3h 7m 56s
INFO:__main__:Best Score: 0.9999


[7.1750581e-04 3.2527730e-04 9.9048135e-04 ... 9.9934632e-01 9.9957401e-01
 9.9946934e-01]


INFO:__main__:========== CV ==========
INFO:__main__:CV: 0.9999


In [ ]:
# OOF
#train_copy.to_csv(ARGS['OUT_DIR'] + f'oof.csv', index=False)
train_copy.to_csv(f"{ARGS['OUT_DIR']}/oof_CV{scores:.4f}.csv", index=False)
train_copy.to_csv(f"{ARGS['DATA_DIR']}/oof_CV{scores:.4f}.csv", index=False)

In [ ]:
import os
os.environ['oof_CVSroce'] = f"{scores:.4f}"

In [ ]:
!ls

drive  sample_data  test  test.zip  train  train.zip


テストデータについて推論を行う

In [ ]:
#sample_submit.csvを読み込みます
#import pandas as pd
#submit = pd.read_csv(f"{ARGS['DATA_DIR']}/sample_submit.csv", header=None)
#submit = pd.read_csv(f"/content/drive/MyDrive/SIGNATE/Sense/sample_submit.csv", header=['image_path', 'label'])
#submit = pd.read_csv(f"/content/drive/MyDrive/SIGNATE/Sense/sample_submit.csv", header=None, names=['image_path', 'label'])
test = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/output/test.csv", header=None, names=['image_path'])
submit = pd.read_csv(f"/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/output/sample_submit.csv", header=None, names=['image_path', 'label'])

test.head()
#submit.head()

,,image_path
file_name,file_path,label
h5XVoGHSnLaKuBzdSFF7BE.jpg,/content/test/h5XVoGHSnLaKuBzdSFF7BE.jpg,-1
JgSjeYnCZngSHseedxpyNj.jpg,/content/test/JgSjeYnCZngSHseedxpyNj.jpg,-1
2bf9zPAzZEyuBhAed4Ysng.jpg,/content/test/2bf9zPAzZEyuBhAed4Ysng.jpg,-1
GCY7soam6ttHjY7prS6XYc.jpg,/content/test/GCY7soam6ttHjY7prS6XYc.jpg,-1


In [ ]:
# test用のデータ拡張
image_transform_test = albu.Compose([
    albu.Resize(ARGS["image_size"][0], ARGS["image_size"][1]),
    albu.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    AT.ToTensorV2 ()
    ])
test_dataset = CustomDataset(submit, image_transform_test, data_type="test")
#test_dataset = CustomDataset2(submit, image_transform_test, data_type="test", is_blurry=ARGS['is_blurry'])
test_loader = DataLoader(test_dataset, batch_size=ARGS["test_batch_size"], shuffle=False, num_workers=1) # 4
#

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()

    dataset_size = 0
    running_loss = 0.0

    predict_list = []

    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, images in bar:
        images = images.to(device)
        with torch.no_grad():
            outputs = model(images)
            #出力にソフトマックス関数を適用
            predicts = outputs.softmax(dim=1)

        predicts = predicts.cpu().detach().numpy()
        predict_list.append(predicts)
    predict_list = np.concatenate(predict_list, axis=0)
    #予測値が1である確率を提出します。
    predict_list = predict_list[:, 1]
    gc.collect()

    return predict_list

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    ARGS['pretrained'] = False
    for i, path in enumerate(model_paths):
        #model = create_model(ARGS)
        model = create_model_timm(ARGS)
        model = model.to(device)

        #学習済みモデルの読み込み
        model.load_state_dict(torch.load(path))
        model.eval()

        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)

    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [ ]:
print(f"{ARGS['OUT_DIR']}")

/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/output


In [ ]:
MODEL_PATHS = [
    f"{ARGS['OUT_DIR']}/Score-Fold-{i}.bin" for i in range(ARGS["n_fold"])
]

推論

In [ ]:
predict_list = inference(MODEL_PATHS, test_loader, ARGS["device"])

Getting predictions for model 1


100%|██████████| 2144/2144 [26:46<00:00,  1.33it/s]


Getting predictions for model 2


100%|██████████| 2144/2144 [26:47<00:00,  1.33it/s]


Getting predictions for model 3


100%|██████████| 2144/2144 [26:46<00:00,  1.33it/s]


Getting predictions for model 4


100%|██████████| 2144/2144 [26:46<00:00,  1.33it/s]


Getting predictions for model 5


100%|██████████| 2144/2144 [26:46<00:00,  1.33it/s]


提出データの作成

In [ ]:
submit['label'] = predict_list
submit.to_csv("/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/output/submission_vit.csv")

#submit['label'] = (predict_list > 0.5)
#submit['label'] = submit['label'].astype(int)
submit.head()

In [ ]:
submit['label'] = predict_list
submit.to_csv("/content/drive/MyDrive/Colab Notebooks/signate_nihonbuyou/output/submission_vit.csv")

In [ ]:
submit['label'] = predict_list


submit['label'] = (predict_list > 0.5)
submit['label'] = submit['label'].astype(int)
submit.head()


submit.to_csv(f"{ARGS['OUT_DIR']}/submission_CV{scores:.4f}.csv", index=False, header=None)